In [2]:
import numpy as np
import nrrd
import pydicom
from pydicom import dcmread
import os
import numpy
from os import listdir
from os.path import isfile, join
import pandas as pd
from ipywidgets import interact
import matplotlib.pyplot as plt
from PIL import Image as im
from matplotlib import pyplot, cm
#Ideas from: https://pynrrd.readthedocs.io/en/latest/examples.html

In [3]:
target_folder = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Duke-Breast-Cancer-MRI\Breast_MRI_190\01-01-1990-NA-MRI BREAST BILATERAL WWO-57539\600.000000-ax 3d dyn-08498"
mypath = target_folder
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
filepaths = [target_folder + "/" + s for s in onlyfiles]

mypath = target_folder
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
testdict = {}
sorteddict = {}
for x in onlyfiles: # for loop to make a dictionary with all the filenames
    tempfile = mypath + "/" + str(x)
    testds = dcmread(tempfile)
    position = testds.ImagePositionPatient[2]
    testdict.update({tempfile: position})

    #then put them in another dictionary ordered by z-coordinate by first retrieving the key and value for the highest value and then using a while loop to each time add the highest value to the sorted dictionary and then removing it from the first 
highestkey = max(testdict, key=testdict.get)
highestvalue = testdict[highestkey]       
while len(testdict) > 0:
    highestkey = max(testdict, key=testdict.get)
    highestvalue = testdict[highestkey]
    sorteddict.update({highestkey: highestvalue})
    testdict.pop(highestkey)

filetitles = list(sorteddict.keys())
RefDs = pydicom.read_file(filetitles[0])
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(filetitles))
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [32]:
def Make_mask(patientnr):
    df = pd.read_excel (r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Annotation_Boxes.xlsx')
    test = df.iloc[patientnr]
    maskstats = pd.Series(test).values
    ArrayMask = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

    for x in range (maskstats[3],maskstats[4]):
        for y in range (maskstats[1],maskstats[2]):
            for z in range (maskstats[5],maskstats[6]+1):
                ArrayMask[x,y,z] = 1
    return(ArrayMask)

In [18]:
testmask = Make_mask(0)
np.all(testmask==0)
print(testmask)
np.where(testmask == 1)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


(array([234, 234, 234, ..., 270, 270, 270], dtype=int64),
 array([308, 308, 308, ..., 340, 340, 340], dtype=int64),
 array([ 89,  90,  91, ..., 110, 111, 112], dtype=int64))

In [33]:
ArrayMask = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)
for patientnr in range (0,922):
    finalmask = Make_mask(patientnr)
    target_location = r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset/Box_masks/'
    target_file = target_location + "mask" + str(patientnr + 1) + ".nrrd"
    data = finalmask
    nrrd.write(target_file, data)

In [ ]:
    df = pd.read_excel (r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Annotation_Boxes.xlsx')
    test = df.iloc[patientnr]
    maskstats = pd.Series(test).values
    ArrayMask = numpy.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

    for x in range (maskstats[3],maskstats[4]):
        for y in range (maskstats[1],maskstats[2]):
            for z in range (maskstats[5],maskstats[6]+1):
                ArrayMask[x,y,z] = 1
    target_location = r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset/Box_masks/'
    target_file = target_location + "mask" + str(patientnr + 1) + ".nrrd"
    data = finalmask
    nrrd.write(target_file, data)

In [7]:
onlymasks = [f for f in listdir(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks") if isfile(join(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks", f))]
#adding mask to mask filenames!!!!!!!
for mask in onlymasks:
    maskterm = "mask" + mask
    targetmask = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks/" + mask
    resultmask = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks/" + maskterm
    os.rename(targetmask, resultmask)

In [31]:
onlymasks = [f for f in listdir(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks") if isfile (join(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks", f))]
findfolders = [f for f in listdir(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds")]
targetfolders = [r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds" + "/" + s for s in findfolders]

for mask in onlymasks:
    masktitle = mask.removesuffix(".nrrd")
    masknumber = masktitle.removeprefix("mask")
    numberamount = len(masknumber)
    
    if numberamount == 1:
        patientidentifier = "00" + masknumber
        for folder in targetfolders:
            if patientidentifier in folder:
                originalpath = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks/" + mask
                targetpath = folder + "/" + mask
                os.replace(originalpath, targetpath)
        
    if numberamount == 2:
        patientidentifier = "0" + masknumber
        for folder in targetfolders:
            if patientidentifier in folder:
                originalpath = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks/" + mask
                targetpath = folder + "/" + mask
                os.replace(originalpath, targetpath)
        
    if numberamount == 3:
        patientidentifier = masknumber
        for folder in targetfolders:
            if patientidentifier in folder:
                originalpath = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Box_masks/" + mask
                targetpath = folder + "/" + mask
                os.replace(originalpath, targetpath)
    

In [6]:
readdata, header = nrrd.read(r"E:\Internship Mass storage\First 3 datasets\VALDO dataset\Test\sub-101\masksub-101 lesion13.nrrd")

print(readdata.shape)
print(header)
print(np.where(readdata == 1))
np.all(readdata==0)

(180, 256, 256)
OrderedDict([('type', 'double'), ('dimension', 3), ('sizes', array([180, 256, 256])), ('endian', 'little'), ('encoding', 'gzip')])
(array([61, 61, 62, 62], dtype=int64), array([117, 118, 117, 118], dtype=int64), array([131, 131, 131, 131], dtype=int64))


False

In [24]:
maskfolders = [f for f in listdir(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds")]
for patientnr in range (52,55): #check range for fina lversion
    patientfolder_found = False
    for folder in maskfolders:
        if patientfolder_found == False:
            print("folder")
            patientlocation = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds/" + folder
            patientfolder = [f for f in listdir(patientlocation)]
            onedigit = "_00"
            twodigit = "_0"
            for file in patientfolder:
                if "image" in file:
                    print("image")
                    if onedigit in folder:
                        patientid2 = folder.removeprefix("Breast_MRI_00")
                        patientid = patientid2.removesuffix(".nrrd")
                        print("1digit")
                        print(patientid)
                        print(patientnr)
                    elif twodigit in folder:
                        patientid2 = folder.removeprefix("Breast_MRI_0")
                        patientid = patientid2.removesuffix(".nrrd")
                        print("2digit")
                        print(patientid)
                        print(patientnr)
                    else:
                        patientid2 = folder.removeprefix("Breast_MRI_")
                        patientid = patientid2.removesuffix(".nrrd")
                        print("3digit")
                        print(patientid)
                        print(patientnr)
                    readdata, header = nrrd.read(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds/" + folder + "/" + file)
                    masksize = readdata.shape
                    print("datashape")
                    patientfind = str(patientnr + 1)
                    if patientid == patientfind:
                        print("if statement works")
                        df = pd.read_excel (r'E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Annotation_Boxes.xlsx')
                        test = df.iloc[patientnr]
                        maskstats = pd.Series(test).values
                        finalmask = numpy.zeros(masksize, dtype=RefDs.pixel_array.dtype)
                        print(masksize)
                        for x in range (maskstats[3],maskstats[4]):
                            for y in range (maskstats[1],maskstats[2]):
                                for z in range (maskstats[5],maskstats[6]):
                                    finalmask[x,y,z] = 1


                        target_location = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds/" + folder + "/"
                        target_file = target_location + "mask" + str(patientnr + 1) + ".nrrd"
                        data = finalmask
                        nrrd.write(target_file, data)
                        patientfolder_found = True
                        print("done")
        
    

folder
image
1digit
1
52
datashape
folder
image
1digit
2
52
datashape
folder
image
1digit
3
52
datashape
folder
image
1digit
4
52
datashape
folder
image
1digit
5
52
datashape
folder
image
1digit
6
52
datashape
folder
image
1digit
7
52
datashape
folder
image
1digit
8
52
datashape
folder
image
1digit
9
52
datashape
folder
image
2digit
10
52
datashape
folder
image
2digit
11
52
datashape
folder
image
2digit
12
52
datashape
folder
image
2digit
13
52
datashape
folder
image
2digit
14
52
datashape
folder
image
2digit
15
52
datashape
folder
image
2digit
16
52
datashape
folder
image
2digit
17
52
datashape
folder
image
2digit
18
52
datashape
folder
image
2digit
19
52
datashape
folder
image
2digit
20
52
datashape
folder
image
2digit
21
52
datashape
folder
image
2digit
22
52
datashape
folder
image
2digit
23
52
datashape
folder
image
2digit
24
52
datashape
folder
image
2digit
25
52
datashape
folder
image
2digit
26
52
datashape
folder
image
2digit
27
52
datashape
folder
image
2digit
28
52
datashape
f

IndexError: index 81 is out of bounds for axis 2 with size 67

In [25]:
maskfolders = [f for f in listdir(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds")]
for folder in maskfolders:
    patientlocation = r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds/" + folder
    patientfolder = [f for f in listdir(patientlocation)]
    for file in patientfolder:
        if "image" in file:    
            readdata, header = nrrd.read(r"E:\Internship Mass storage\First 3 datasets\Duke_breast_MRI dataset\Converted\converted_nrrds/" + folder + "/" + file)
            masksize = readdata.shape
            print(masksize)

(448, 448, 160)
(512, 512, 60)
(512, 512, 64)
(256, 256, 90)
(448, 448, 160)
(512, 512, 53)
(448, 448, 208)
(448, 448, 160)
(512, 512, 58)
(512, 512, 52)
(512, 512, 62)
(512, 512, 70)
(512, 512, 60)
(448, 448, 176)
(448, 448, 112)
(512, 512, 58)
(448, 448, 176)
(448, 448, 208)
(320, 320, 160)
(256, 256, 96)
(512, 512, 60)
(448, 448, 160)
(448, 448, 192)
(512, 512, 71)
(512, 512, 68)
(448, 448, 208)
(448, 448, 192)
(448, 448, 208)
(512, 512, 192)
(512, 512, 82)
(512, 512, 65)
(512, 512, 60)
(512, 512, 64)
(512, 512, 60)
(512, 512, 56)
(512, 512, 64)
(320, 320, 144)
(512, 512, 72)
(448, 448, 160)
(448, 448, 160)
(512, 512, 60)
(448, 448, 192)
(512, 512, 38)
(448, 448, 128)
(512, 512, 62)
(448, 448, 256)
(256, 256, 72)
(448, 448, 176)
(512, 512, 36)
(512, 512, 192)
(256, 256, 100)
(448, 448, 128)
(512, 512, 67)
(512, 512, 36)
(512, 512, 60)
(512, 512, 40)
(448, 448, 112)
(256, 256, 90)
(512, 512, 62)
(512, 512, 64)
(512, 512, 58)
(512, 512, 58)
(448, 448, 176)
(512, 512, 35)
(512, 512, 60

(384, 384, 60)
(448, 448, 176)
(256, 256, 98)
(512, 512, 38)
(512, 512, 70)
(448, 448, 160)
(512, 512, 68)
(512, 512, 60)
(256, 256, 60)
(512, 512, 64)
(512, 512, 73)
(448, 448, 160)
(448, 448, 160)
(448, 448, 176)
(512, 512, 58)
(512, 512, 64)
(512, 512, 192)
(512, 512, 57)
(512, 512, 64)
(512, 512, 96)
(512, 512, 192)
(448, 448, 144)
(512, 512, 192)
(512, 512, 66)
(512, 512, 61)
(512, 512, 64)
(448, 448, 160)
(512, 512, 55)
(512, 512, 71)
(448, 448, 208)
(256, 256, 64)
(448, 448, 120)
(512, 512, 68)
(512, 512, 49)
(512, 512, 60)
(448, 448, 160)
(448, 448, 144)
(448, 448, 144)
(512, 512, 192)
(512, 512, 69)
(512, 512, 60)
(512, 512, 200)
(512, 512, 36)
(512, 512, 60)
(256, 256, 56)
(448, 448, 160)
(512, 512, 192)
(448, 448, 192)
(512, 512, 70)
(448, 448, 160)
(512, 512, 76)
(512, 512, 59)
(512, 512, 55)
(512, 512, 62)
(448, 448, 208)
(512, 512, 68)
(448, 448, 176)
(512, 512, 30)
(256, 256, 65)
(512, 512, 53)
(512, 512, 62)
(512, 512, 42)
(448, 448, 160)
(448, 448, 60)
(512, 512, 65)
(